In [1]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import math

import matplotlib.pylab as plt
import plotly.express as px
from torchvision import transforms, datasets
from torch.utils.data import DataLoader,TensorDataset,Dataset
import torchvision

Cargamos el dataset

In [2]:
iris=pd.read_csv('https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv')

In [3]:
iris['species'].unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

Definimos nuestra red neuronal

In [4]:
class red_neuronal(nn.Module):
  def __init__(self, nin,middle,nout) :
    super(red_neuronal,self).__init__()

    self.fc1=nn.Linear(nin,middle)
    self.fc2=nn.Linear(middle,nout)
    self.softmax=nn.Softmax(dim=1)
  def forward(self,x):
    out=self.fc1(x)
    out=self.fc2(out)
    out=self.softmax(out)
    return out

Cambiamos los nombres de las clases

In [5]:
iris['species']=iris['species'].map({'setosa':0,'versicolor':1,'virginica':2})

Definimos entradas y salidas

In [6]:
X=iris[[i for i in iris.columns if i not in ['species']]]
y=iris['species'].values

In [ ]:
iris

Convertimos a tensores y creamos el dataset


In [7]:
X=torch.tensor(np.array(X),dtype=torch.float)
y=torch.tensor(y)
dataset=TensorDataset(X,y)

Entrenamiento y prueba

In [8]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

Creamos la estructura para que pueda ser leida por el modelo

In [9]:
traindataloader=DataLoader(train_dataset,shuffle=True,batch_size=8)
testdataloader=DataLoader(test_dataset,shuffle=True,batch_size=8)

LLamamos nuestro modelo, en este caso nuestra red neuronal tiene 4 entradas y 3 salidas. También vemos que se configuró una tasa de aprendizaje de 0.001 ( puedes probar diferentes tasas para ver como se comporta)

In [10]:
neural_network=red_neuronal(4,16,3)
lossfn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(neural_network.parameters(),lr=0.001)

Entrenamiento ( puedes variar el número de epocas para ver que pasa)

In [11]:
epochs=100
loss_ep_train=[]
acc_ep_train=[]
loss_ep_test=[]
acc_ep_test=[]
max_acc=0
for e in range(epochs):
  corrects=0
  loss_run_train=[]
  loss_acum_t=0.0

  for x,y in traindataloader:
    z=neural_network(x)
    loss=lossfn(z,y)
    _,preds=torch.max(z,1)
    corrects+=torch.sum(preds==y).item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss_run_train.append(loss.item())
    loss_acum_t+=loss.item()
  acctrain=corrects/len(traindataloader.dataset)
  acc_ep_train.append(acctrain)
  #loss_ep_train.append(np.mean(loss_run_train))
  loss_ep_train.append(loss_acum_t/len(traindataloader.dataset))
  neural_network.eval()
  corrects=0
  loss_run_test=[]
  loss_acum_te=0.0
  for x,y in testdataloader:
    z=neural_network(x)
    loss=lossfn(z,y)
    _,preds=torch.max(z,1)
    corrects+=torch.sum(preds==y).item()
    loss_run_test.append(loss.item())
    loss_acum_te+=loss.item()
  acctest=corrects/len(testdataloader.dataset)
  acc_ep_test.append(acctest)
  #loss_ep_test.append(np.mean(loss_run_test))
  loss_ep_test.append(loss_acum_te/len(testdataloader.dataset))
  print(f'epoch: {e+1}/{epochs}')
  if acctest>max_acc:
    best_model=neural_network
    max_acc=acctest
    print("Encontré un mejor modelo!!")
    torch.save(neural_network.state_dict(),f'mejor_modelo.pt')
    print('Nuevo mejor modelo!')
  #print(f'[train] error: {round(np.mean(loss_run_train),2)} accuracy: {round(acctrain,2)}')
  #print(f'[test] error: {round(np.mean(loss_run_test),2)} accuracy: {round(acctest,2)}')
  print(f'[train] error: {loss_acum_t/len(traindataloader.dataset)} accuracy: {round(acctrain,2)}')
  print(f'[test] error: {loss_acum_te/len(testdataloader.dataset)} accuracy: {round(acctest,2)}')


epoch: 1/100
Encontré un mejor modelo!!
Nuevo mejor modelo!
[train] error: 0.13031082202990849 accuracy: 0.42
[test] error: 0.15353318055470785 accuracy: 0.1
epoch: 2/100
[train] error: 0.127769768734773 accuracy: 0.39
[test] error: 0.154324205716451 accuracy: 0.1
epoch: 3/100
[train] error: 0.1259942149122556 accuracy: 0.39
[test] error: 0.14944632450739542 accuracy: 0.1
epoch: 4/100
Encontré un mejor modelo!!
Nuevo mejor modelo!
[train] error: 0.1235727330048879 accuracy: 0.41
[test] error: 0.14355351130167643 accuracy: 0.3
epoch: 5/100
Encontré un mejor modelo!!
Nuevo mejor modelo!
[train] error: 0.12085418154795964 accuracy: 0.64
[test] error: 0.13925569852193195 accuracy: 0.63
epoch: 6/100
[train] error: 0.11805661022663116 accuracy: 0.67
[test] error: 0.13351110021273296 accuracy: 0.63
epoch: 7/100
[train] error: 0.11545654932657877 accuracy: 0.68
[test] error: 0.12735759218533835 accuracy: 0.63
epoch: 8/100
[train] error: 0.11293553610642751 accuracy: 0.68
[test] error: 0.125368

ploteamos la accuracy para los datos de entrenamiento y de prueba,( aqui vemos ocmo el modelo fue aprendiendo a traves del tiempo ya que su accuracy fue aumentando)

In [12]:
import plotly.graph_objects as go
fig=go.Figure()
fig.add_trace(go.Scatter(y=acc_ep_train,name='train'))
fig.add_trace(go.Scatter(y=acc_ep_test,name='test'))
fig.update_xaxes(title='epocas')
fig.update_yaxes(title='Accuracy')
fig.show()

ploteamos el  error para los datos de entrenamiento y de prueba,( aqui vemos como el modelo fue aprendiendo a traves del tiempo ya que su error fue disminuyendo)

In [13]:
fig=go.Figure()
fig.add_trace(go.Scatter(y=loss_ep_train,name='train'))
fig.add_trace(go.Scatter(y=loss_ep_test,name='test'))
fig.update_xaxes(title='epocas')
fig.update_yaxes(title='Error')
fig.show()

finalmente vemos el desempeño final con los datos de prueba

In [14]:
finaltestdataloader=DataLoader(test_dataset,shuffle=True,batch_size=len(test_dataset))
for xtest,ytrue in finaltestdataloader:
  pass
_,preds=torch.max(best_model(xtest),1)

In [15]:
from sklearn.metrics import classification_report
print(classification_report(ytrue.numpy(), preds.numpy(),target_names=['Iris-setosa','Iris-versicolor','Iris-virginica']))

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        16
Iris-versicolor       1.00      0.91      0.95        11
 Iris-virginica       0.75      1.00      0.86         3

       accuracy                           0.97        30
      macro avg       0.92      0.97      0.94        30
   weighted avg       0.97      0.97      0.97        30

